In [37]:
import tensorflow as tf
import keras


config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 10} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [38]:
import numpy as np
import os
import re
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.utils import to_categorical
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [39]:
dirname = os.path.join(os.getcwd(), 'trainprueba')
imgpath = dirname + os.sep 
 
images = []
directories = []
dircount = []
prevRoot=''
cant=0
 
print("leyendo imagenes de ",imgpath)
 
for root, dirnames, filenames in os.walk(imgpath):
    for filename in filenames:
        if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
            cant=cant+1
            filepath = os.path.join(root, filename)
            image = plt.imread(filepath)
            images.append(image)
            b = "Leyendo..." + str(cant)
            print (b, end="\r")
            if prevRoot !=root:
                print(root, cant)
                prevRoot=root
                directories.append(root)
                dircount.append(cant)
                cant=0
dircount.append(cant)
 
dircount = dircount[1:]
dircount[0]=dircount[0]+1
print('Directorios leidos:',len(directories))
print("Imagenes en cada directorio", dircount)
print('suma Total de imagenes en subdirs:',sum(dircount))

leyendo imagenes de  C:\Users\UsX\Documents\GitHub\Tetris-Neural-Network\image\trainprueba\
C:\Users\UsX\Documents\GitHub\Tetris-Neural-Network\image\trainprueba\I 1
C:\Users\UsX\Documents\GitHub\Tetris-Neural-Network\image\trainprueba\J 89
C:\Users\UsX\Documents\GitHub\Tetris-Neural-Network\image\trainprueba\L 64
C:\Users\UsX\Documents\GitHub\Tetris-Neural-Network\image\trainprueba\O 74
C:\Users\UsX\Documents\GitHub\Tetris-Neural-Network\image\trainprueba\S 75
C:\Users\UsX\Documents\GitHub\Tetris-Neural-Network\image\trainprueba\T 110
C:\Users\UsX\Documents\GitHub\Tetris-Neural-Network\image\trainprueba\Z 80
Directorios leidos: 7
Imagenes en cada directorio [90, 64, 74, 75, 110, 80, 79]
suma Total de imagenes en subdirs: 572


In [40]:
labels=[]
indice=0
for cantidad in dircount:
    for i in range(cantidad):
        labels.append(indice)
    indice=indice+1
print("Cantidad etiquetas creadas: ",len(labels))

blocks=[]
indice=0
for directorio in directories:
    name = directorio.split(os.sep)
    print(indice , name[len(name)-1])
    blocks.append(name[len(name)-1])
    indice=indice+1

y = np.array(labels)
X = np.array(images, dtype=np.uint8) #convierto de lista a numpy

# Find the unique numbers from the train labels
classes = np.unique(y)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Cantidad etiquetas creadas:  572
0 I
1 J
2 L
3 O
4 S
5 T
6 Z
Total number of outputs :  7
Output classes :  [0 1 2 3 4 5 6]


In [41]:
#Mezclar todo y crear los grupos de entrenamiento y testing
train_X,test_X,train_Y,test_Y = train_test_split(X,y,test_size=0.2)
print('Training data shape : ', train_X.shape, train_Y.shape)
print('Testing data shape : ', test_X.shape, test_Y.shape)

train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X = train_X / 255.
test_X = test_X / 255.

# Change the labels from categorical to one-hot encoding
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)

# Display the change for category label using one-hot encoding
print('Original label:', train_Y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])

train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, random_state=13)

print(train_X.shape,valid_X.shape,train_label.shape,valid_label.shape)

Training data shape :  (457, 100, 100, 3) (457,)
Testing data shape :  (115, 100, 100, 3) (115,)
Original label: 3
After conversion to one-hot: [0. 0. 0. 1. 0. 0. 0.]
(365, 100, 100, 3) (92, 100, 100, 3) (365, 7) (92, 7)


In [42]:
INIT_LR = 1e-3
epochs = 512
batch_size = 512

block_model = Sequential()
block_model.add(Conv2D(64, kernel_size=(4, 4),activation='linear',padding='same',input_shape=(100,100,3)))
block_model.add(LeakyReLU(alpha=0.1))
block_model.add(MaxPooling2D((4, 4),padding='same'))
block_model.add(Dropout(0.5))

block_model.add(Flatten())
block_model.add(Dense(64, activation='linear'))
block_model.add(LeakyReLU(alpha=0.1))
block_model.add(Dropout(0.5)) 
block_model.add(Dense(nClasses, activation='softmax'))

block_model.summary()

block_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adagrad(lr=INIT_LR, decay=INIT_LR / 100),metrics=['accuracy'])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 100, 100, 64)      3136      
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 100, 100, 64)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 25, 25, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 25, 25, 64)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 40000)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 64)                2560064   
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 64)               

In [43]:
block_train_dropout = block_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))

# guardamos la red, para reutilizarla en el futuro, sin tener que volver a entrenar
block_model.save("block_mnist.h5py")

Train on 365 samples, validate on 92 samples
Epoch 1/512
365/365 [==============================] - 1s 2ms/step - loss: 1.9459 - acc: 0.1178 - val_loss: 1.9372 - val_acc: 0.1957
Epoch 2/512
365/365 [==============================] - 0s 430us/step - loss: 1.9382 - acc: 0.2027 - val_loss: 1.9281 - val_acc: 0.1957
Epoch 3/512
365/365 [==============================] - 0s 400us/step - loss: 1.9278 - acc: 0.2082 - val_loss: 1.9199 - val_acc: 0.1957
Epoch 4/512
365/365 [==============================] - 0s 397us/step - loss: 1.9247 - acc: 0.2027 - val_loss: 1.9181 - val_acc: 0.1957
Epoch 5/512
365/365 [==============================] - 0s 403us/step - loss: 1.9167 - acc: 0.2027 - val_loss: 1.9123 - val_acc: 0.1957
Epoch 6/512
365/365 [==============================] - 0s 403us/step - loss: 1.9163 - acc: 0.2027 - val_loss: 1.9089 - val_acc: 0.1957
Epoch 7/512
365/365 [==============================] - 0s 406us/step - loss: 1.8988 - acc: 0.2027 - val_loss: 1.9004 - val_acc: 0.1957
Epoch 8/512


Epoch 61/512
365/365 [==============================] - 0s 419us/step - loss: 1.7168 - acc: 0.3315 - val_loss: 1.6992 - val_acc: 0.4239
Epoch 62/512
365/365 [==============================] - 0s 425us/step - loss: 1.7233 - acc: 0.3863 - val_loss: 1.6978 - val_acc: 0.5435
Epoch 63/512
365/365 [==============================] - 0s 403us/step - loss: 1.7124 - acc: 0.4603 - val_loss: 1.6946 - val_acc: 0.4239
Epoch 64/512
365/365 [==============================] - 0s 403us/step - loss: 1.7141 - acc: 0.3507 - val_loss: 1.6911 - val_acc: 0.5435
Epoch 65/512
365/365 [==============================] - 0s 406us/step - loss: 1.7075 - acc: 0.4192 - val_loss: 1.6867 - val_acc: 0.5435
Epoch 66/512
365/365 [==============================] - 0s 411us/step - loss: 1.7013 - acc: 0.4548 - val_loss: 1.6857 - val_acc: 0.4239
Epoch 67/512
365/365 [==============================] - 0s 406us/step - loss: 1.7043 - acc: 0.3644 - val_loss: 1.6801 - val_acc: 0.5435
Epoch 68/512
365/365 [==========================

365/365 [==============================] - 0s 403us/step - loss: 1.5081 - acc: 0.7315 - val_loss: 1.4779 - val_acc: 0.6196
Epoch 122/512
365/365 [==============================] - 0s 417us/step - loss: 1.5115 - acc: 0.6438 - val_loss: 1.4763 - val_acc: 0.6196
Epoch 123/512
365/365 [==============================] - 0s 417us/step - loss: 1.5110 - acc: 0.6658 - val_loss: 1.4726 - val_acc: 0.6196
Epoch 124/512
365/365 [==============================] - 0s 414us/step - loss: 1.5088 - acc: 0.6685 - val_loss: 1.4676 - val_acc: 0.6196
Epoch 125/512
365/365 [==============================] - 0s 400us/step - loss: 1.4986 - acc: 0.6493 - val_loss: 1.4646 - val_acc: 0.6196
Epoch 126/512
365/365 [==============================] - 0s 408us/step - loss: 1.4944 - acc: 0.6603 - val_loss: 1.4605 - val_acc: 0.6196
Epoch 127/512
365/365 [==============================] - 0s 408us/step - loss: 1.4990 - acc: 0.6384 - val_loss: 1.4567 - val_acc: 0.6196
Epoch 128/512
365/365 [==============================] 

Epoch 181/512
365/365 [==============================] - 0s 403us/step - loss: 1.2892 - acc: 0.8685 - val_loss: 1.2392 - val_acc: 0.8043
Epoch 182/512
365/365 [==============================] - 0s 406us/step - loss: 1.2840 - acc: 0.7973 - val_loss: 1.2342 - val_acc: 1.0000
Epoch 183/512
365/365 [==============================] - 0s 406us/step - loss: 1.2763 - acc: 0.8466 - val_loss: 1.2297 - val_acc: 0.8043
Epoch 184/512
365/365 [==============================] - 0s 403us/step - loss: 1.2678 - acc: 0.8521 - val_loss: 1.2282 - val_acc: 0.8478
Epoch 185/512
365/365 [==============================] - 0s 406us/step - loss: 1.2916 - acc: 0.8247 - val_loss: 1.2270 - val_acc: 0.8043
Epoch 186/512
365/365 [==============================] - 0s 408us/step - loss: 1.2598 - acc: 0.8438 - val_loss: 1.2239 - val_acc: 1.0000
Epoch 187/512
365/365 [==============================] - 0s 414us/step - loss: 1.2703 - acc: 0.8356 - val_loss: 1.2188 - val_acc: 0.8043
Epoch 188/512
365/365 [==================

Epoch 241/512
365/365 [==============================] - 0s 411us/step - loss: 1.1064 - acc: 0.8986 - val_loss: 1.0370 - val_acc: 1.0000
Epoch 242/512
365/365 [==============================] - 0s 411us/step - loss: 1.1091 - acc: 0.8685 - val_loss: 1.0369 - val_acc: 1.0000
Epoch 243/512
365/365 [==============================] - 0s 403us/step - loss: 1.1331 - acc: 0.8822 - val_loss: 1.0329 - val_acc: 1.0000
Epoch 244/512
365/365 [==============================] - 0s 406us/step - loss: 1.0960 - acc: 0.9041 - val_loss: 1.0303 - val_acc: 1.0000
Epoch 245/512
365/365 [==============================] - 0s 411us/step - loss: 1.0837 - acc: 0.9041 - val_loss: 1.0260 - val_acc: 1.0000
Epoch 246/512
365/365 [==============================] - 0s 414us/step - loss: 1.0978 - acc: 0.9068 - val_loss: 1.0233 - val_acc: 1.0000
Epoch 247/512
365/365 [==============================] - 0s 411us/step - loss: 1.0914 - acc: 0.8932 - val_loss: 1.0207 - val_acc: 1.0000
Epoch 248/512
365/365 [==================

Epoch 301/512
365/365 [==============================] - 0s 414us/step - loss: 0.9882 - acc: 0.9233 - val_loss: 0.8861 - val_acc: 1.0000
Epoch 302/512
365/365 [==============================] - 0s 414us/step - loss: 0.9525 - acc: 0.9096 - val_loss: 0.8847 - val_acc: 1.0000
Epoch 303/512
365/365 [==============================] - 0s 403us/step - loss: 0.9695 - acc: 0.9342 - val_loss: 0.8818 - val_acc: 1.0000
Epoch 304/512
365/365 [==============================] - 0s 403us/step - loss: 0.9643 - acc: 0.9151 - val_loss: 0.8809 - val_acc: 1.0000
Epoch 305/512
365/365 [==============================] - 0s 406us/step - loss: 0.9450 - acc: 0.9178 - val_loss: 0.8771 - val_acc: 1.0000
Epoch 306/512
365/365 [==============================] - 0s 406us/step - loss: 0.9788 - acc: 0.9014 - val_loss: 0.8767 - val_acc: 1.0000
Epoch 307/512
365/365 [==============================] - 0s 411us/step - loss: 0.9592 - acc: 0.9205 - val_loss: 0.8744 - val_acc: 1.0000
Epoch 308/512
365/365 [==================

Epoch 361/512
365/365 [==============================] - 0s 411us/step - loss: 0.8631 - acc: 0.9397 - val_loss: 0.7634 - val_acc: 1.0000
Epoch 362/512
365/365 [==============================] - 0s 417us/step - loss: 0.8491 - acc: 0.9342 - val_loss: 0.7626 - val_acc: 1.0000
Epoch 363/512
365/365 [==============================] - 0s 414us/step - loss: 0.8478 - acc: 0.9233 - val_loss: 0.7604 - val_acc: 1.0000
Epoch 364/512
365/365 [==============================] - 0s 408us/step - loss: 0.8602 - acc: 0.9260 - val_loss: 0.7585 - val_acc: 1.0000
Epoch 365/512
365/365 [==============================] - 0s 419us/step - loss: 0.8526 - acc: 0.9288 - val_loss: 0.7559 - val_acc: 1.0000
Epoch 366/512
365/365 [==============================] - 0s 411us/step - loss: 0.8265 - acc: 0.9342 - val_loss: 0.7536 - val_acc: 1.0000
Epoch 367/512
365/365 [==============================] - 0s 403us/step - loss: 0.8672 - acc: 0.9315 - val_loss: 0.7509 - val_acc: 1.0000
Epoch 368/512
365/365 [==================

Epoch 421/512
365/365 [==============================] - 0s 417us/step - loss: 0.7505 - acc: 0.9425 - val_loss: 0.6529 - val_acc: 1.0000
Epoch 422/512
365/365 [==============================] - 0s 422us/step - loss: 0.7528 - acc: 0.9562 - val_loss: 0.6518 - val_acc: 1.0000
Epoch 423/512
365/365 [==============================] - 0s 417us/step - loss: 0.7518 - acc: 0.9616 - val_loss: 0.6497 - val_acc: 1.0000
Epoch 424/512
365/365 [==============================] - 0s 414us/step - loss: 0.7485 - acc: 0.9479 - val_loss: 0.6493 - val_acc: 1.0000
Epoch 425/512
365/365 [==============================] - 0s 411us/step - loss: 0.7453 - acc: 0.9507 - val_loss: 0.6466 - val_acc: 1.0000
Epoch 426/512
365/365 [==============================] - 0s 406us/step - loss: 0.7519 - acc: 0.9534 - val_loss: 0.6464 - val_acc: 1.0000
Epoch 427/512
365/365 [==============================] - 0s 411us/step - loss: 0.7397 - acc: 0.9397 - val_loss: 0.6449 - val_acc: 1.0000
Epoch 428/512
365/365 [==================

Epoch 481/512
365/365 [==============================] - 0s 406us/step - loss: 0.6763 - acc: 0.9479 - val_loss: 0.5694 - val_acc: 1.0000
Epoch 482/512
365/365 [==============================] - 0s 408us/step - loss: 0.6786 - acc: 0.9589 - val_loss: 0.5670 - val_acc: 1.0000
Epoch 483/512
365/365 [==============================] - 0s 414us/step - loss: 0.6799 - acc: 0.9534 - val_loss: 0.5656 - val_acc: 1.0000
Epoch 484/512
365/365 [==============================] - 0s 408us/step - loss: 0.6819 - acc: 0.9507 - val_loss: 0.5640 - val_acc: 1.0000
Epoch 485/512
365/365 [==============================] - 0s 411us/step - loss: 0.6700 - acc: 0.9644 - val_loss: 0.5615 - val_acc: 1.0000
Epoch 486/512
365/365 [==============================] - 0s 415us/step - loss: 0.6652 - acc: 0.9671 - val_loss: 0.5579 - val_acc: 1.0000
Epoch 487/512
365/365 [==============================] - 0s 427us/step - loss: 0.6497 - acc: 0.9534 - val_loss: 0.5541 - val_acc: 1.0000
Epoch 488/512
365/365 [==================

In [44]:
test_eval = block_model.evaluate(test_X, test_Y_one_hot, verbose=1)

print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

115/115 [==============================] - 0s 244us/step
Test loss: 0.5519261401632558
Test accuracy: 0.991304347826087


In [45]:
test_eval

[0.5519261401632558, 0.991304347826087]

In [46]:
im = [plt.imread('trainprueba/Z/_1.png')]
X = np.array(im, dtype=np.uint8)
prediction = block_model.predict(X)
print(prediction)

[[0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 1.3558646e-08]]


In [47]:
"""
import pyautogui
import time

def screenshot(delay, name):
    time.sleep(delay)
    screenshot = pyautogui.screenshot()
    screenshot.save("capturas/_"+str(name)+".png")

cont = 233
while True:
    screenshot(0.5, cont)
    cont+=1
"""

'\nimport pyautogui\nimport time\n\ndef screenshot(delay, name):\n    time.sleep(delay)\n    screenshot = pyautogui.screenshot()\n    screenshot.save("capturas/_"+str(name)+".png")\n\ncont = 233\nwhile True:\n    screenshot(0.5, cont)\n    cont+=1\n'